# SQL ANALYSIS

In [1]:
import csv, sqlite3
import prettytable
import pandas as pd
prettytable.DEFAULT = 'DEFAULT'
%load_ext sql
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

### To directly operate on the data, the .csv file has been provided in this repository named 'Spacexsql.csv'

In [2]:
%sql sqlite:///my_data1.db

In [3]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql('SPACEXTBL',if_exists='replace',con=con,index=False,method='multi')

101

In [4]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [5]:
%sql create table SPACEXTBL as select * from SPACEXTBL where Data is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTBL already exists
[SQL: create table SPACEXTBL as select * from SPACEXTBL where Data is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Display the names of the unique launch sites in the space mission¶

### Display 5 records where launch sites begin with the string 'CCA'

In [6]:
%sql select * from SPACEXTBL where Launch_site like 'CCA%' limit 5 

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Display the total payload mass carried by boosters launched by NASA (CRS)

In [7]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where Customer like 'NASA (CRS)%'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
48213


### Display average payload mass carried by booster version F9 v1.1

In [8]:
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where Booster_Version like 'F9 v1.1%'

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2534.6666666666665


### List the date when the first succesful landing outcome in ground pad was acheived

In [9]:
%sql select min(Date) from SPACEXTBL where Landing_Outcome like'Success%'

 * sqlite:///my_data1.db
Done.


min(Date)
2015-12-22


### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [12]:
%sql select distinct(Booster_Version) from SPACEXTBL where (Landing_Outcome like 'Success (drone ship)%' and (PAYLOAD_MASS__KG_ between 4000 and 6000))

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### List the total number of successful and failure mission outcomes

In [13]:
%sql select total(Mission_Outcome like 'Success'),total(Mission_Outcome like 'Failure%') from SPACEXTBL

 * sqlite:///my_data1.db
Done.


total(Mission_Outcome like 'Success' ),total(Mission_Outcome like 'Failure%' )
98.0,1.0


### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function

In [18]:
%sql select distinct(Booster_Version),(select max(PAYLOAD_MASS__KG_)) from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Booster_Version,(select max(PAYLOAD_MASS__KG_))
F9 B5 B1048.4,15600


### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
Note: SQLLite does not support monthnames. So you need to use substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.

In [39]:
%sql select substr(Date,6,2),Landing_Outcome,Booster_Version,Launch_Site from Spacextbl where substr(Date,0,5)='2015' and Landing_Outcome like 'Failure (drone ship)%'

 * sqlite:///my_data1.db
Done.


"substr(Date,6,2)",Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [ ]:
%sql select count(Landing_Outcome) as Outcomecount from spacextbl where Date between '2010-06-04' and '2017-03-20' group by 'Landing_Outcome' order by Outcomecount DESC
# Not the exact outcome as expected because it requires multiple sub-queries at once.


 * sqlite:///my_data1.db
Done.


Outcomecount
31


In [13]:
%sql SELECT Booster_Version FROM spacextbl WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM spacextbl)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


# END